In [3]:
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [4]:
is_python = 'inline' in matplotlib.get_backend()
if is_python: from IPython import display


# Create DQN Class, Epsilon Greedy Strategy, Replay Memory and Agent Class

In [5]:
class DQN(nn.Module):
    def _init_(self, img_height, img_width):
        super().__init__()

        self.fc1 = nn.Linear(in_features = img_height * img_width * 3, out_features = 24)           # The "*3" corresponds to the three recieved color channels
        self.fc2 = nn.Linear(in_features = 24, out_features = 32)
        self.out = nn.Linear(in_features = 32, out_features = 2)            # Our only options here are either to move left or to move right (2d cart)

    def forward(self,t):
        t = t.flatten(start_dim=1)          # any initial input t must first be formatted. This gives us a ONE dimensional tensor (remember that's a matrix! and 0 dim is a vector)
        t = F.relu(self.fc1(t))         # Pass it through the activation functions
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t


In [9]:
    Experience = namedtuple(         # Create a tuple class named 'experience' (kinda like the self. objects of a class but without the methods that come with them)
        'Experience',
        ('state', 'action', 'next_state', 'reward')
    )


In [10]:
e = Experience(2,3,1,4)

In [11]:
e

Experience(state=2, action=3, next_state=1, reward=4)

In [12]:
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
    
    def push(self, experience):
        if len(self.memory) < self.capacity:            # If the memory so far hasn't hit the capacity
            self.memory.append(experience)
        else:
            self.memory[self.push % self.capacity] = experience     # The "leftover" (modulus) of push you're on by the total storage
        self.push_count += 1

    def sample(self, batch_size):           # grab batch_size number of samples from memory to train the network
        return random.sample(self.memory, batch_size)

    def can_provide_sample(self, batch_size):           # Do we have enough experiences to sample n_batchsize to train?
        return len(self.memory) >= batch_size


In [13]:
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay

    def get_exploration_rate(self, current_step):           # Adds on top of the min (most likely to exploit), the range times an exponentially decreasing value
        return self.end + (self.start - self.end) * \           
            math.exp(-1. * current_step * self.decay)


In [17]:
class Agent():
    def __init__(self, strategy, num_actions):
        self.current_step = 0           # Initializes at position 0
        self.strategy = strategy            # This is our epsilon greedy number (threshhold)
        self.num_actions = num_actions          # How many possible actions can an agent make from a given state

    def select_action(self, state, policy_net):
        rate = strategy.get_exploration_rate(self.current_step)         # Generate the random number that tells us either to explore or exploit
        self.current_step += 1

        if rate > random.random():
            return random.randrange(self.num_actions)           # Give a random action from the ones available for that state
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim= 1).item()

# Main Program

In [ ]:
batch_size = 256
gamma = 0.999
eps_start = 1
eps_end = 0.01
eps_decay = 0.001  
target_update = 10          # Update the target network (as opposed to the policy network) every 10 moves
memory_size = 100000
lr = 0.001              # lr of the training of the policy network
num_episodes = 1000


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
em = CartPoleEnvManager(device)         # I have no idea what this does
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)         # Creates the strategy(ie threshhold)
agent = Agent(strategy, em.num_actions_available(), device)         # Initializes our agen
memory = ReplayMemory(memory_size)

policy_net = DQN(em.get_screen_height(), em.get_screen_width()).to(device)          # The "to" means run it "to" our device
target_net = DQN(em.get_screen_height(), em.get_screen_width()).to(device)